In [11]:
import numpy as np

In [9]:
# 读取文件数据
def LoadData(file):
    # 打开文件
    fr = open(file)
    
    dataArr = []# 数据集
    classLabel = []# 标签集
    #读取每一行的数据
    for line in fr.readlines():
        data = line.strip().split('\t')
        dataArr.append(data[:2])
        classLabel.append(data[2])
    return dataArr,classLabel

In [3]:
def KernelTrans(X1,X2,kTup):
    m = np.shape(X1)[0]#行数
    K = np.mat(np.zeros(m,1))
    if kTup[0] == "lin":#核函数是xi*xj.T的积
        K = X1*X2.T
    elif kTup[0] == "rbg":#核函数是径向基
        for i in range(m):
            deltaRow = X1[i,:] - A
            K[i] = deltaRow*deltaRow.T
        K = np.exp(K/(-1*kTup[1]**2))
    return K

In [4]:
class opStruct():
    def __init__(self,dataArr,classLabel,C,maxIter,toler,kTup):
        self.X = dataArr#数据集
        self.classLabel = classLabel#标签集
        self.C = C#
        self.maxIter = maxIter#最大次数
        self.toler = toler
        self.m = np.shape(dataArr)#数据集的行数
        self.alphas = np.mat(np.zeros((m,1)))
        self.b = 0
        self.eCache = np.mat(np.zeors(m,2))
        self.K = np.mat(np.zeros(np.m,np.m))
        for i in range(self.m):
            #注意这里，K的一列，是核函数的结果
            self.K[:,i] = KernelTrans(self.X,self.X[i,:],kTup)
        

In [12]:
#计算Ei
def calcE(op,i):
    fki = float(np.multiply(op.alphas,op.classLabel.T).T*op.K[:,i])+op.b
    Ei = fki - float(np.classLabel[i])
    return Ei

In [ ]:
#随机选择第二个alpha[j]
def selectJRand(op,i):
    j = i
    
    #两个alpha不能相同
    while(j==i)
        j = int(np.random.uniform(0,m))

In [15]:
#启发式选择第二个alpha[j]
def selectJ(op,Ei,i):
    maxDelta = -1;maxJ = -1;Ej
    op.eCache[i] = [1,Ei]
    #选择eCache中不为0的alpha
    valist = np.nonzero(op.eCache[:,0].A>0)[0]
    
    #选择|Ei-Ej|最大的第二个alpha[j]
    if len(valist) >0:
        for k in valist:
            if k==i:
                continue
            Ej = calcE(op,k)
            delta = abs(Ei-Ej)
            if delta<maxDelta:
                maxDelta = delta
                maxJ = k
    else:
        #随机选择第二个alpha[j]
        Ej,maxJ = selectJRand(op,i)
    
    return Ej,maxJ 

In [ ]:
def clipAlpha(alpha,L,H):
    if alpha < L:
        alpha = L
    elif alpha > H:
        alpha = H
    return alpha

In [18]:
#更新alpha[i]的Ei
def updateEk(op,i):
    Ei = calcE(op,i)
    op.eCache[i] = [1,Ei]

In [24]:
#根据已经选择的第一个alpha[i]，再选择第二个alpha[j],对alpha和b进行更新
def innerl(op,i):
    #计算当前选择的alpha[i]的Ei
    Ei = calcE(op,i)
    
    #选择第二个alpha[j]的启发式方法，使|Ei-Ej|最大
    #如果Ei*yi<0，则选择最大的Ex为Ej
    #如果Ej*yj>0,则选择最小的Ex为Ej
    if(Ei*op.classLabel[i]<-op.toler and op.alphas[i]<op.C or 
        Ei*op.classLabel[i]>op.toler and op.alphas[i]>0):
        Ej,j = selectJ(op,Ei,i)
        alphaOldI = op.alphas[i].copy()
        alphaOldJ = op.alphas[j].copy()
        
        #L<alpha<H,y[i]与y[j]的符号不同有不同的上下界
        if op.classLabel[i]!=op.classLabel[j] :
            L = max(0,op.alphas[j]-op.alphas[i])
            H = min(op.C,op.C+op.alphas[j]-op.alphas[i])
        else:
            L = max(0,op.alphas[i]+op.alphas[j]-op.C)
            H = min(op.C,op.alphas[j]+op.alphas[i])
        
        #更新第二个alpha[j]和Ej
        op.alphas[j] = op.alphaOldJ[j]+ op.classLabel[j]*(Ei - Ej)/(op.K[i,i]+op.K[j,j]-2*op.K[i,k])
        op.alphas[j] = clipAlpha(op.alphas[j],L,H)
        
        updateEk(op,j)
        
        #如果变化的值很小
        if(abs(op.alphas[j]-alphaOldJ)<0.00001):
            print("j not move enough")
        
        #更新alpha[i]还有Ei
        op.alphas[i] += op.classLabel[i]*op.classLabel[j]*(alphaOldJ-op.alphas[j])
        updateEk(op,i)
        
        b1 = op.b -Ei - op.classLabel[i]*(op.alphas[i]-alphaOldI)*op.K[i,i]-op.classLabel[j]*(op.alphas[j] - alphaOldJ)*op.K[i,j]
        
        b2 = op.b - Ej - op.classLabel[i]*(op.alphas[i]-alphaOldI)*op.K[i,j]- op.classLabel[j]*(op.classLabel[j]-alphaOldJ)*op.K[j,j]
        
        if 0 < op.alphas[i] and op.alphas[i] < op.C:
            op.b = b1
        elif o<op.alphas[j] and op.alphas[j] < op.C:
            op.b = b2
        else:
            op.b = (b1+b2)/2.0
        return 1
    return 0

In [8]:
#SMO算法
def SMO(dataArr,classLable,C,maxIter,kTup=['lin',0]):
    op = opStruct(dataArr,classLabel,C,maxIter,kTup)
    flag = True#表示是否遍历全部alpha
    iter = 0
    alphaChanged = 0
    
    #迭代次数小于最大次数并且有改变alpha 或者 要求遍历整个alpha集改变
    while((iter<maxIter and alphaChanged>0) or flag == True):
        #遍历整个alpha集，更新alpha
        if flag == true:
            for i in range(op.m):
                alphaChanged += innerl(op,i)
                print("fullSet,iter:%d i:%d,pairs changed:%d"%(iter,i,alphaChanged))
            
            #迭代次数+1
            iter += 1
        else:
            #更新0<alpha<C中的alpha
            #获取0<alpha<C的行坐标
            nonBounds = np.nonzero((op.alphas.A>0)*(op.alphas.A<op.C))[0]
            
            #根据alpha[i]选择alpha[j]
            for i in nonBounds:
                alphaChanged += innerl(op,i)
                print("nonBonds,iter:%d i:%d ,pairs changed %d"%(iter,i,alphaChanged))
            
            #迭代次数+1
            iter += 1
         
        if flag:
            flag = false
        elif alphaChanged == 0:
            flag = true
        
        print("iteration number %d"%iter)
        return op.alphas,op.b
         


In [10]:
print(LoadData("testSet.txt"))

([['3.542485', '1.977398'], ['3.018896', '2.556416'], ['7.551510', '-1.580030'], ['2.114999', '-0.004466'], ['8.127113', '1.274372'], ['7.108772', '-0.986906'], ['8.610639', '2.046708'], ['2.326297', '0.265213'], ['3.634009', '1.730537'], ['0.341367', '-0.894998'], ['3.125951', '0.293251'], ['2.123252', '-0.783563'], ['0.887835', '-2.797792'], ['7.139979', '-2.329896'], ['1.696414', '-1.212496'], ['8.117032', '0.623493'], ['8.497162', '-0.266649'], ['4.658191', '3.507396'], ['8.197181', '1.545132'], ['1.208047', '0.213100'], ['1.928486', '-0.321870'], ['2.175808', '-0.014527'], ['7.886608', '0.461755'], ['3.223038', '-0.552392'], ['3.628502', '2.190585'], ['7.407860', '-0.121961'], ['7.286357', '0.251077'], ['2.301095', '-0.533988'], ['-0.232542', '-0.547690'], ['3.457096', '-0.082216'], ['3.023938', '-0.057392'], ['8.015003', '0.885325'], ['8.991748', '0.923154'], ['7.916831', '-1.781735'], ['7.616862', '-0.217958'], ['2.450939', '0.744967'], ['7.270337', '-2.507834'], ['1.749721', '-